selenium, bs4를 설치해 브라우저를 원격 조정하자.

컴퓨터가 최소 8GB 이상, i5 이상의 CPU, Windows 환경을 추천한다.

In [ ]:
!pip install selenium

In [ ]:
!pip install bs4

Firefox를 설치하자.

다음으로 geckodriver를 설치해야 한다.

https://github.com/mozilla/geckodriver/releases 에서 본인에게 맞는 OS를 골라 설치하자.

압축을 풀고 C:/Windows 폴더에 geckodriver.exe를 복사한다. 이때 관리자 권한이 필요하다. 리눅스나 mac사용자는 /usr/bin 으로 geckodriver를 옮기면 된다.

예를 들어, Linux 사용자는 터미널에서
```
    $ wget https://github.com/mozilla/geckodriver/releases/download/v0.27.0/geckodriver-v0.27.0-linux64.tar.gz
    $ sudo sh -c 'tar -x geckodriver -zf geckodriver-v0.27.0-linux64.tar.gz -O > /usr/bin/geckodriver'
    $ sudo chmod +x /usr/bin/geckodriver
    $ rm geckodriver-v0.73.0-linux64.tar.gz
```



In [1]:
from selenium import webdriver #웹드라이브
from selenium.webdriver.firefox.options import Options #Firefox options
from selenium.webdriver.common.keys import Keys #key emulation
from bs4 import BeautifulSoup as BS

In [2]:
import time

In [3]:
'''
Factory - Firefox
'''
def fire_fox_browser(headless=False,image_allow=True):
    options=Options()
    profile=webdriver.FirefoxProfile()
    profile.set_preference('permissions.default.image',1 if image_allow else 2)
    options.headless=headless
    driver=webdriver.Firefox(firefox_profile=profile,options=options)
    return driver

In [4]:
browser=fire_fox_browser(image_allow=False) #이미지를 표시하지 않는다. 렌더링을 하지 않아 속도에서 이득
#browser=fire_fox_browser(True) #headless mode로 열린다. 즉, 브라우저 창은 실제로 보이지 않게 된다.

In [5]:
youtube_url="https://www.youtube.com/watch?v=9kU6jb8XXpw" #펭TV 남극 동물의 한국식 여름 휴가, 코멘트 많은
youtube_url_no_comment="https://www.youtube.com/watch?v=S2qJAO03SsU" #코멘트 없는
youtube_url_few_comment="https://www.youtube.com/watch?v=-5WjyN53Zw4" #코멘트 적은

In [6]:
browser.get(youtube_url)

Key END로 화면을 스크롤 다운시킨다. 그리고 스크롤 이후 content 업데이트가 추가로 있는지 확인하기 위해 Javascript 실행하여 결과를 비교한다.

In [7]:
def press_down(h,tb):
    h.send_keys(Keys.END)
    time.sleep(tb)
'''
driver: 웹브라우저
time_buff: 스크롤 체크 시간
trial: 더 이상 콘텐츠가 있는지 확인하기 위한 control code
'''
def scroll_down(driver,time_buff=0.5):
    initial_height = driver.execute_script("return window.scrollY")
    html=driver.find_element_by_tag_name('html')
    press_down(html,time_buff)
    press_down(html,time_buff)
    last_height = driver.execute_script("return window.scrollY")
    if initial_height == last_height:
        return False
    while True:
        time.sleep(time_buff)
        new_height = driver.execute_script("return window.scrollY")
        if new_height==last_height:
            break
        last_height=new_height
    return True

In [8]:
def remove_element_by_id(b,s):
    b.execute_script("document.getElementById('{}').remove();".format(s))

In [9]:
def remove_comments(b):
    script='''
    document.querySelectorAll("#replies").forEach(function(item,index){item.remove();});
    var reviews=document.querySelectorAll('#body');
    reviews.forEach(function(item,index){
        if(index<(reviews.length-1)) item.remove();
    })
    '''
    b.execute_script(script)

In [10]:
def reply_more(b,tb=1):
    eles=b.find_elements_by_xpath('//yt-formatted-string[contains(.,"View")]')
    for ele in eles:
        if ele.text is not "":
            ele.click()
    time.sleep(tb)
    eles=b.find_elements_by_xpath('//yt-formatted-string[contains(.,"Show more replies")]')
    for ele in eles:
        if ele.text is not "":
            ele.click()
    time.sleep(tb)   

script='''
    var eles=document.querySelectorAll('[slot*="more"]');
    eles.forEach(function(item,index){item.click();});
    '''
    b.execute_script(script)

In [11]:
def text_more(b,tb=1):
    eles=b.find_elements_by_xpath('//span[contains(.,"Read more")]')
    for ele in eles:
        if ele.text is not "":
            ele.click()
    time.sleep(tb)

In [12]:
def collect_comments(b):
    page=b.page_source

In [13]:
def test(u):
    browser.get(u)
    time.sleep(2)
    remove_element_by_id(browser,"related")
    time.sleep(1)
    remove_element_by_id(browser,'movie_player')
    time.sleep(1)
    total_comment=[]
    while True:
        if scroll_down(browser,time_buff=2) is False:
            print('break')
            break
        reply_more(browser)
        reply_more(browser)
        reply_more(browser)
        text_more(browser)
        page=browser.page_source
        soup=BS(page,'html.parser')
        total_comment+=[ele.text for ele in soup.select('#content-text')]
        remove_comments(browser)    
        print(len(total_comment))
    return total_comment

In [ ]:
comments=test(youtube_url_few_comment)

69


In [36]:
import pandas as pd

In [88]:
pd.Series(comments).to_clipboard()

In [38]:
comments[21]

'안때리실겁니다\n원하시면 넣어보는거지요'

In [90]:
text_more(browser)